# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [15]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import time
# Import API key
from config import geo_api

In [3]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("Resources/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,faya,18.3851,42.4509,22.06,35,21,2.60,SA,1666108228
1,1,farsund,58.0948,6.8047,13.30,100,0,7.65,NO,1666108228
2,2,new norfolk,-42.7826,147.0587,11.72,58,12,1.34,AU,1666108230
3,3,jamestown,42.0970,-79.2353,5.77,77,100,9.77,US,1666107934
4,4,lanzhou,36.0564,103.7922,14.53,48,59,1.20,CN,1666108230


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [4]:
%%capture --no-display

# Configure the map plot
city_data_df.hvplot.points(
    x='Lng',  
    y='Lat',   
    size='Humidity',  
    geo=True,
    tiles='OSM',  
    frame_width=700,
    frame_height=500
)


:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [5]:
# Narrow down cities that fit criteria and drop any results with null values
columns = ["City_ID", "City", "Lat", "Lng", "Max Temp", "Humidity", "Cloudiness", "Wind Speed", "Country"]

ideal_city_df = city_data_df.loc[city_data_df["Cloudiness"] == 0, columns]

# Drop any rows with null values
ideal_city_df.fillna(0, inplace= True)

# Display sample data
ideal_city_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country
1,1,farsund,58.0948,6.8047,13.30,100,0,7.65,NO
7,7,carnarvon,-24.8667,113.6333,20.02,76,0,7.46,AU
9,9,saint-pierre,-21.3393,55.4781,22.82,68,0,5.14,RE
32,32,cape town,-33.9258,18.4232,20.61,71,0,9.26,ZA
44,44,collierville,35.0420,-89.6645,7.11,47,0,5.66,US


### Step 3: Create a new DataFrame called `hotel_df`.

In [11]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
columns = ["City", "Country", "Lat", "Lng", "Humidity"]
hotel_df = city_data_df[columns]
hotel_df.head()
# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df.loc[:, "Hotel Name"] = ""

# Display sample data
hotel_df.head()

C:\Users\Carl\AppData\Local\Temp\ipykernel_14084\2993631659.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df.loc[:, "Hotel Name"] = ""


,City,Country,Lat,Lng,Humidity,Hotel Name
0,faya,SA,18.3851,42.4509,35,
1,farsund,NO,58.0948,6.8047,100,
2,new norfolk,AU,-42.7826,147.0587,58,
3,jamestown,US,42.0970,-79.2353,77,
4,lanzhou,CN,36.0564,103.7922,48,


In [ ]:
hotel_df

### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [17]:
# Set parameters to search for a hotel
radius = 10000

# Define parameters for the hotel search
params = {
    "categories": "hotel",
    "radius": radius,
    "apiKey": geo_api  
}
# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"

    # Set base URL for the Geoapify Places API
    base_url = "https://api.geoapify.com/v2/places"

    # Make an API request using the params dictionary
    response = requests.get(base_url, params=params)

    # Get the JSON response data
    hotel_data = response.json()

# Extract the name and address of the first hotel found
if hotel_data["features"]:
    first_hotel = hotel_data["features"][0]
    name_address = first_hotel["properties"]["name"] + ", " + first_hotel["properties"]["address"]
else:
    name_address = "No hotels found within 10,000 meters"

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
try:
    hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
except (KeyError, IndexError):
    # If no hotel is found, set the hotel name as "No hotel found".
    hotel_df.loc[index, "Hotel Name"] = "No hotel found"
# pause to avoid rate limiting
time.sleep(1)
# Log the search res
# ults
print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")
# Display sample data
hotel_df

Starting hotel search


KeyError: 'features'

In [18]:
hotel_data

{'statusCode': 400,
 'error': 'Bad Request',
 'message': 'Invalid parameters. Category "hotel" is not supported. The supported categories are: accommodation.hotel, accommodation.hut, accommodation.apartment, accommodation.chalet, accommodation.guest_house, accommodation.hostel, accommodation.motel, accommodation, activity.community_center, activity.sport_club, activity, adult.nightclub, adult.stripclub, adult.swingerclub, adult.brothel, adult.casino, adult.adult_gaming_centre, adult, airport.private, airport.international, airport.military, airport.gliding, airport.airfield, airport, amenity.toilet, amenity.drinking_water, amenity.give_box.food, amenity.give_box.books, amenity.give_box, amenity, beach.beach_resort, beach, administrative.continent_level, administrative.country_level, administrative.country_part_level, administrative.state_level, administrative.county_level, administrative.city_level, administrative.district_level, administrative.suburb_level, administrative.neighbourhoo

### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [ ]:
%%capture --no-display

# Configure the map plot
# YOUR CODE HERE

# Display the map
# YOUR CODE HERE

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)